In [14]:
from skimage import data as sd
import skimage.color as sc
from skimage.viewer import ImageViewer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

pd.set_option('precision',10)

img = sd.imread('./Both/286279.jpg')
img_hsv = sc.convert_colorspace(img, 'RGB', 'HSV')

viewer = ImageViewer(img)

h = img_hsv.shape[0]
w = img_hsv.shape[1]#(600, 467, 3) height, width, 3 vals for HSV
#viewer.show() #displays image

In [2]:
'''
support function for features 1-3
'''

def make_dataframe(img_hsv):
    height = len(img_hsv)
    width = len(img_hsv[0])
    pixels = height * width
    hue = []
    saturation = []
    value = []
    
    for i in range(height):
        for j in range(width):
            hue.append(img_hsv[i][j][0])
            saturation.append(img_hsv[i][j][1])
            value.append(img_hsv[i][j][2])
            
    data = {'H': hue, 'S': saturation, 'V': value}
    hsv_df = pd.DataFrame(data=data)
    
    return hsv_df

hsv_df = make_dataframe(img_hsv)

In [3]:
'''
the first two features: average hue, average saturation
'''

def f1_2(hsv_df):
    return hsv_df.mean()['H'], hsv_df.mean()['S']

f1, f2 = f1_2(hsv_df)

In [4]:
'''
support function for features 3 to 23, k is number of bins
'''
def make_bins(hsv_df, k):
    bins = [0]
    increment = 1/k
    for i in range(k):
        temp = bins[i] + increment
        bins.append(temp)
    hue_bins = pd.cut(hsv_df['H'], bins=bins, include_lowest=True).value_counts()
    return hue_bins

hue_bins = make_bins(hsv_df, 20)

In [9]:
'''
count the num of hue bins that are most prominent, c is a multiplicatory component
'''
def f3(hue_bins, c):
    
    max_bins = max(hue_bins.tolist())
    count = 0
    
    for i in range(len(hue_bins)):
        if hue_bins[i] > c * max_bins:
            count +=1
            
    return count

f3 = f3(hue_bins, 0.1)
hue_bins

(0.5, 0.55]    132766
(0.55, 0.6]    118700
(0.1, 0.15]     44837
(0.05, 0.1]     33341
(0.15, 0.2]     28703
(0.45, 0.5]     20393
(0.4, 0.45]      8578
(0.2, 0.25]      7173
(0.25, 0.3]      4309
(0.35, 0.4]      4227
(0.3, 0.35]      3343
[0, 0.05]        3195
(0.6, 0.65]       570
(0.95, 1]         477
(0.9, 0.95]       345
(0.65, 0.7]       253
(0.85, 0.9]       114
(0.7, 0.75]       113
(0.75, 0.8]        91
(0.8, 0.85]        72
Name: H, dtype: int64

In [16]:
'''
features 4 to 23, the dispersion of each hue component, returns as list
'''

def f4_23(hue_bins, h, w):
    px = h * w
    hue_bins = hue_bins.sort_index().tolist()
    color_dispersion = []
    for i in range(len(hue_bins)):
        color_dispersion.append(hue_bins[i]/px)
          
    return color_dispersion

f4_23 = f4_23(hue_bins, h, w)

In [ ]:
from scipy import ndimage

ndimage.measurements.center_of_mass(img_hsv)

In [ ]:
from scipy.cluster.vq import vq, kmeans

def do_cluster(hsv_image, K, channels):
    # gets height, width and the number of channes from the image shape
    h,w,c = hsv_image.shape
    # prepares data for clustering by reshaping the image matrix into a (h*w) x c matrix of pixels
    cluster_data = hsv_image.reshape( (h*w,c) )
    # grabs the initial time
#     t0 = t.time()
    # performs clustering
    codebook, distortion = kmeans(np.array(cluster_data[:,0:channels], dtype=np.float), K)
    # takes the final time
#     t1 = t.time()
#     print "Clusterization took %0.5f seconds" % (t1-t0)
    
    
    # calculates the total amount of pixels
    tot_pixels = h*w
    # generates clusters
    data, dist = vq(cluster_data[:,0:channels], codebook)
    # calculates the number of elements for each cluster
    weights = [len(data[data == i]) for i in range(0,K)]
    
    # creates a 4 column matrix in which the first element is the weight and the other three
    # represent the h, s and v values for each cluster
    color_rank = np.column_stack((weights, codebook))
    # sorts by cluster weight
    color_rank = color_rank[np.argsort(color_rank[:,0])]

    # creates a new blank image
    new_image =  np.array([0,0,255], dtype=np.uint8) * np.ones( (500, 500, 3), dtype=np.uint8)
    img_height = new_image.shape[0]
    img_width  = new_image.shape[1]

    # for each cluster
    for i,c in enumerate(color_rank[::-1]):
        
        # gets the weight of the cluster
        weight = c[0]
        
        # calculates the height and width of the bins
        height = int(weight/float(tot_pixels) *img_height )
        width = round(img_width/len(color_rank))

        # calculates the position of the bin
        x_pos = i*width

        # defines a color so that if less than three channels have been used
        # for clustering, the color has average saturation and luminosity value
        color = np.array( [0,128,200], dtype=np.uint8)
        
        # substitutes the known HSV components in the default color
        for j in range(len(c[1:])):
            color[j] = c[j+1]
        
        # draws the bin to the image
        new_image[ img_height-height:img_height, x_pos:x_pos+width] = [color[0], color[1], color[2]]
        
    # returns the cluster representation
    return new_image

# creates a new figure size 12x10 inches
plt.figure(figsize=(12,10))
# creates a 4-column subplot
plt.subplot(141)
# in the first cell draws the target image
plt.imshow(img)

# calculates clusters for
# * h
# * h and s
# * h, s and v

for i in range(1,4):
    plt.subplot(141 + i)
    plt.title("Channels: %i" % i)
    new_image = do_cluster(img_hsv, 5, i)
    new_image = sc.hsv2rgb(new_image)
    plt.imshow(new_image)
    
plt.show()

In [ ]:
from scipy import ndimage as ndi
import matplotlib.pyplot as plt

from skimage.morphology import watershed, disk
from skimage import data
from skimage.filters import rank
from skimage.util import img_as_ubyte



image = img_as_ubyte(sd.imread('./Both/285826.jpg', as_grey=True))

# denoise image
denoised = rank.median(image, disk(2))

# find continuous region (low gradient -
# where less than 10 for this image) --> markers
# disk(5) is used here to get a more smooth image
markers = rank.gradient(denoised, disk(5)) < 10
markers = ndi.label(markers)[0]

# local gradient (disk(2) is used to keep edges thin)
gradient = rank.gradient(denoised, disk(2))

# process the watershed
labels = watershed(gradient, markers)

# display results
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8, 8), sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'})
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray, interpolation='nearest')
ax[0].set_title("Original")

ax[1].imshow(gradient, cmap=plt.cm.spectral, interpolation='nearest')
ax[1].set_title("Local Gradient")

ax[2].imshow(markers, cmap=plt.cm.spectral, interpolation='nearest')
ax[2].set_title("Markers")

# ax[3].imshow(image, cmap=plt.cm.gray, interpolation='nearest')
ax[3].imshow(labels, cmap=plt.cm.spectral, interpolation='nearest', alpha=.7)
ax[3].set_title("Segmented")

for a in ax:
    a.axis('off')

fig.tight_layout()
plt.show()

labels[:][5]